1\. **Maximum wind speed prediction at the Sprogø station**

The exercise goal is to predict the maximum wind speed occurring every 50 years even if no measure exists for such a period. The available data are only measured over 21 years at the Sprogø meteorological station located in Denmark. 

The annual maxima are supposed to fit a normal probability density function. However such function is not going to be estimated because it gives a probability from a wind speed maxima. Finding the maximum wind speed occurring every 50 years requires the opposite approach, the result needs to be found from a defined probability. That is the quantile function role and the exercise goal will be to find it. In the current model, it is supposed that the maximum wind speed occurring every 50 years is defined as the upper 2% quantile.

By definition, the quantile function is the inverse of the cumulative distribution function. The latter describes the probability distribution of an annual maxima. In the exercise, the cumulative probability $p_i$ for a given year i is defined as $p_i = i/(N+1)$ with $N = 21$, the number of measured years. Thus it will be possible to calculate the cumulative probability of every measured wind speed maxima. From those experimental points, the scipy.interpolate module will be very useful for fitting the quantile function. Finally the 50 years maxima is going to be evaluated from the cumulative probability of the 2% quantile.

Practically, load the dataset:

```python
import numpy as np
max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
```

Compute then the cumulative probability $p_i$ (`cprob`) and sort the maximum speeds from the data. Use then the  UnivariateSpline from scipy.interpolate to define a quantile function and thus estimate the probabilities.

In the current model, the maximum wind speed occurring every 50 years is defined as the upper 2% quantile. As a result, the cumulative probability value will be:

```python
fifty_prob = 1. - 0.02
```

So the storm wind speed occurring every 50 years can be guessed as:

``` python
fifty_wind = quantile_func(fifty_prob)
```



In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

max_speeds = np.load('max-speeds.npy')
years_nb = max_speeds.shape[0]
years = np.linspace(1,21,21)
cprob = [i/(years_nb+1) for i in years]
max_speeds = np.sort(max_speeds)
plt.plot(max_speeds, cprob,'.')

quantile_func = sp.interpolate.UnivariateSpline(cprob, max_speeds)
plt.plot(quantile_func(cprob),cprob,'-')

fifty_prob = 1. - 0.02
fifty_wind = quantile_func(fifty_prob)

2\. **Curve fitting of temperature in Alaska** 

The temperature extremes in Alaska for each month, starting in January, are given by (in degrees Celcius):

max:  17,  19,  21,  28,  33,  38, 37,  37,  31,  23,  19,  18

min: -62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58

* Plot these temperature extremes.
* Define a function that can describe min and max temperatures. 
* Fit this function to the data with scipy.optimize.curve_fit().
* Plot the result. Is the fit reasonable? If not, why?
* Is the time offset for min and max temperatures the same within the fit accuracy?

In [ ]:
from scipy import optimize

max_temp = np.array([17, 19, 21, 28, 33, 38, 37, 37, 31, 23, 19, 18])
min_temp = np.array([-62, -59, -56, -46, -32, -18, -9, -13, -25, -46, -52, -58])

# plot temperature extremes
months = np.linspace( 1, 12, 12 )
plt.scatter( months, max_temp, color='red' )
plt.scatter( months, min_temp, color='blue' )

# functionthat can describe min and max temperatures
def gaussian(x, a, x0, sigma, off):
    return a*np.exp(-(x-x0)**2. / (2 * sigma**2.))+ off

def f(times, avg, ampl, time_offset):
    return (avg + ampl * np.cos((times + time_offset) * 2 * np.pi / times.max()))

rM, cM = optimize.curve_fit( f, months, max_temp )
rm, cm = optimize.curve_fit( gaussian, months, min_temp )

days = np.linspace(0, 12, num=365)
plt.plot(days, f( days, *rM ), color='red', linestyle='dashed', label='$f$ function fitting' )
plt.plot(days, gaussian( days, *rm ), color='blue', linestyle='dashed', label='Gaussian function fitting' )
plt.xlabel('Encoded month')
plt.ylabel('Temperature')
plt.title('Alaska Temperature Fitting')
plt.legend( fontsize=9, )
plt.show()

3\. **2D minimization of a six-hump camelback function**

$$
f(x,y) = \left(4-2.1x^2+\frac{x^4}{3} \right) x^2 +xy + (4y^2 -4)y^2
$$

has multiple global and local minima. Find the global minima of this function.

Hints:

* Variables can be restricted to $-2 < x < 2$ and $-1 < y < 1$.
* Use numpy.meshgrid() and pylab.imshow() to find visually the regions.
* Use scipy.optimize.minimize(), optionally trying out several of its methods.

How many global minima are there, and what is the function value at those points? What happens for an initial guess of $(x, y) = (0, 0)$ ?


In [ ]:
N = 100

def f(x,y):
    return ( 4 - 2.1 * x**2 + x**4/3 ) * x**2 + x * y + ( 4 * y**2 - 4) * y**2

x = np.linspace(-2,2,N)
y = np.linspace(-1,1,N)

x, y = np.meshgrid(x, y)

plt.figure()
plt.imshow( f(x, y), extent=[-2, 2, -1, 1] )
plt.colorbar()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(20, 10))

ax = fig.gca(projection='3d')
ax.plot_surface( x, y, f(x, y) )

plt.show()

In [ ]:
import matplotlib.cm as cm

plt.imshow( f(x,y), origin='lower', extent=[-2, 2, -1, 1] )
plt.contour( x, y, f(x,y), cmap=cm.viridis, levels=np.arange(-2,6,0.15))
plt.show()

In [ ]:
# saddle point in (0,0)

def param_f(t):
    x,y = t
    return ( 4 - 2.1 * x**2 + x**4/3 ) * x**2 + x * y + ( 4 * y**2 - 4) * y**2

p = [ 0, 0 ]
min_ = result = optimize.minimize( param_f, p )
print( min_.x )
print( param_f(min_.x) )

4\. **FFT of a simple dataset**

Performe a periodicity analysis on the lynxs-hares population

In [ ]:
from scipy import fftpack
from scipy import signal

df = np.loadtxt( 'populations.txt' )
year, hares, lynxes, carrots = df.T

plt.plot( year, hares, year, lynxes, year, carrots ) 
plt.legend( ('Hare', 'Lynx', 'Carrot') ) 

fig = plt.figure( "FFT", figsize=(20,4) )
fig2 = plt.figure( "Peak" )
c = 1

for i in ( hares, lynxes, carrots ):
    plt.figure( 'FFT' )
    fft = fftpack.fft( i )
    abs_fft = np.abs( fft )
    freq = fftpack.fftfreq( i.size )
    
    ax1 = fig.add_subplot( 1, 3, c )
    if (c==1):
        plt.title('Hares FFT power')
    elif (c==2):
        plt.title('Lynxes FFT power')
    else:
        plt.title('Carrot FFT power')
    
    ax1 = plt.plot( freq, abs_fft )
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    
    mask = np.where(freq > 0)
    
    l_freq = freq[mask]
    l_peak = l_freq[ abs_fft[mask].argmax() ]
    
    if (c==1):
        leg = 'Hares'
        print('Hares peak freq: ', l_peak )
    elif (c==2):
        leg = 'Lynxes'
        print('Hares peak freq: ', l_peak )
    else:
        leg = 'Carrot'
        print('Hares peak freq: ', l_peak )

    hf_fft = fftpack.fft(i)
    hf_fft[np.abs(freq)>l_peak] = 0 
    f_ifft = fftpack.ifft(hf_fft)
    plt.figure('OrigVsFilt'+leg)
    if (c==1):
        plt.title('Hares FFT power')
    elif (c==2):
        plt.title('Lynxes FFT power')
    else:
        plt.title('Carrot FFT power')
    plt.plot(year, i, label='Original signal')
    plt.plot(year, f_ifft, linewidth=3, label='Filtered signal')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.legend(loc='best')

    c+=1
plt.figure('Peak')
plt.legend(bbox_to_anchor=(1,1))

5\. **FFT of an image**

* Examine the provided image `moonlanding.png`, which is heavily contaminated with periodic noise. In this exercise, we aim to clean up the noise using the Fast Fourier Transform.
* Load the image using pylab.imread().
* Find and use the 2-D FFT function in scipy.fftpack, and plot the spectrum (Fourier transform of) the image. Do you have any trouble visualising the spectrum? If so, why?
* The spectrum consists of high and low frequency components. The noise is contained in the high-frequency part of the spectrum, so set some of those components to zero (use array slicing).
* Apply the inverse Fourier transform to see the resulting image.

In [ ]:
import pylab
from matplotlib.colors import LogNorm

# Load image
img = plt.imread('moonlanding.png')
data = plt.imread('moonlanding.png')

fft = fftpack.fft2(data)
power = np.abs(fft)

plt.figure(figsize=(18, 18))
ax1 = plt.subplot(2, 2, 1)
ax1.set_title('Original Image')
ax1.imshow(img)

ax1 = plt.subplot(2, 2, 2)
ax1.set_title('Original Image - grey')
ax1.imshow(img, cmap=plt.cm.gray)

ax2 = plt.subplot(2, 2, 3)
ax2.imshow(power, norm=LogNorm(vmin=5))
ax2.set_title('FFT power log')


# k=the fraction of coefficients (in each direction) we keep
k = 0.1
rows, cols = fft.shape

# Set to zero 
fft[ int(rows*k):int(rows*(1-k)) ] = 0
fft[:, int(cols*k):int(cols*(1-k))] = 0

ax5 = plt.subplot(2, 2, 4)
ax5.set_title('Cleared Image')
cleared = fftpack.ifft2(fft).real 
ax5.imshow(cleared, cmap=plt.cm.gray)

plt.show()
